# Demonstrating the dataset generation for an ampl model

This notebook shows the generation of the dataset that are subsequently used to train a neural network for optimal control.

In [ ]:
### %pylab notebook
from pyquad.ekin import*
from tqdm import tqdm

In [ ]:
ampl_mod_path = "ampl/bebop_6dof.mod"

num = 100000

# INITIAL CONDITIONS
y0 = np.random.uniform(-10.0, 10.0, num)
z0 = np.random.uniform(-10.0, 10.0, num)

vy0 = np.random.uniform(-5.,5., num)
vz0 = np.random.uniform(-5.,5., num)
theta0 = np.random.uniform(-np.pi/3, np.pi/3, num)

omega0 = np.random.uniform(-0.01, -0.01, num)


parameters = {
    "y0"    : y0,
    "z0"    : z0,
    "vy0"   : vy0,
    "vz0"   : vz0,
    "theta0": theta0,
    "omega0": omega0,
}

fixed_parameters = {
    "yn" : 0.0,
    "zn" : 0.0,
    "vyn" : 0.0,
    "vzn" : 0.0,
    "thetan" : 0.0,
    "omegan" : 0.0,  
    "epsilon" : 1.0,  
}

In [ ]:
def run_ampl_model(param_dict, file_path):
    model = AMPLModel(ampl_mod_path)
    model.setParameterValues(param_dict)
    model.setParameterValues(fixed_parameters)
    model.solve()
    soln_vals = model.getSolutionValues()
    obj_vals = model.getObjectiveValues()
    # save solution to file
    np.savez(file_path, **{"Success": model.checkSolved(), "t": soln_vals['timegrid'], **soln_vals, **obj_vals, **param_dict, **fixed_parameters})

In [ ]:
name = '2D_QUAD_HOVER_EPS=0.5'
datafolder = 'datasets/' + name
datafile = 'datasets/' + name + '.npz'

# make folder if it doesn't exist
if not os.path.exists(datafolder):
    os.makedirs(datafolder)

In [ ]:
##### trajectories to solve
index_set = set(range(num)) -  {int(file.replace('.npz','')) for file in os.listdir(datafolder)}

# solve trajectories in parallel
njobs = 20
sol_lst = joblib.Parallel(njobs)(joblib.delayed(run_ampl_model)(
    {key: val[idx] for key,val in parameters.items()},
    datafolder + '/' + str(idx)
) for idx in tqdm(index_set))

print('Generated', num,'trajectories in', datafolder)

In [ ]:
def load_file(file, datafolder):
    try:
        return dict(np.load(datafolder + '/' + file))
    except:
        return None

njobs = 100
trajectory_list = joblib.Parallel(njobs)(
    joblib.delayed(load_file)(file, datafolder) for file in tqdm(os.listdir(datafolder))
)
trajectory_list = [traj for traj in trajectory_list if traj != None]

In [ ]:
keys = trajectory_list[0].keys()
trajectory_list = [traj for traj in trajectory_list if traj.keys()==keys]
keys = keys - {key+'m' for key in keys if key+'m' in keys}

dataset = {key: np.stack([traj[key] for traj in trajectory_list]) for key in tqdm(keys)}

# default parameters
params = AMPLModel(ampl_mod_path).getParameterValues()
dataset = {**params, **dataset}

# save dataset
np.savez_compressed(datafile, **dataset)

**Filter failed trajectories**

In [ ]:
total = len(dataset['Success'])
print('total:', total)

# UNSUCCESSFUL TRAJECTORIES
unsuccessful = np.where(dataset['Success'] == False)[0]
print('unsuccessful:', len(unsuccessful))

# OBJECTIVE TOO HIGH
high_obj = np.where(dataset['myobjective'] > 7.0)[0]
print('high obj:', len(high_obj))
# outlier plot
plt.plot(dataset['myobjective'])
plt.axhline(7.0, color='r')
plt.show()
# yz plot
idx = np.random.choice(list(high_obj), 100)
plt.plot(dataset['y'][idx].T, dataset['z'][idx].T, color='b', alpha=0.1)
plt.show()


# TF TOO HIGH
high_tf = np.where(dataset['tf'] > 7.0)[0]
print('high tf:', len(high_tf))
# outlier plot
plt.plot(dataset['tf'])
plt.axhline(7.0, color='r')
plt.show()
# yz plot
idx = np.random.choice(list(high_tf), 100)
plt.plot(dataset['y'][idx].T, dataset['z'][idx].T, color='b', alpha=0.1)
plt.show()

# THETA TOO HIGH
high_theta = np.where(np.max(np.abs(dataset['theta']), axis=1) > np.pi)[0]
print('high theta:', len(high_theta))
# outlier plot
plt.plot(np.max(np.abs(dataset['theta']), axis=1))
plt.axhline(np.pi, color='r')
plt.show()
# yz plot
idx = np.random.choice(list(high_theta), 100)
plt.plot(dataset['y'][idx].T, dataset['z'][idx].T, color='b', alpha=0.1)
plt.show()

# FILTERED DATASET
discard = set(unsuccessful) | set(high_obj) | set(high_tf) | set(high_theta)
keep = set(range(total)) - discard
print('keep:', len(keep))
# yz plot
idx = np.random.choice(list(keep), 100)
plt.plot(dataset['y'][idx].T, dataset['z'][idx].T, color='b', alpha=0.1)
plt.show()

dataset_filtered = dict()
if discard:
    for key in tqdm(dataset.keys()):
        # check if dataset[key] is an array
        if isinstance(dataset[key], np.ndarray):
            if dataset[key].shape[0] == num:
                dataset_filtered[key] = dataset[key][list(keep)]
        else:
            dataset_filtered[key] = dataset[key]
    np.savez_compressed(datafile, **dataset_filtered)

**View Trajectories**

In [ ]:
# select trajectories with objective higher than 10
indices = keep

if len(indices) > 1000:
    indices = np.random.choice(list(indices), 1000, replace=False)

trajs = [{key: dataset[key][index] for key in ['t', 'y', 'z', 'theta', 'ur', 'ul']} for index in indices]

In [ ]:
from quadcopter_animation import animation2D

animation2D.animate(*trajs)

**Post Processing**

In [ ]:
keys = ['t', 'y', 'z', 'vy', 'vz', 'theta', 'omega', 'ul', 'ur']
indices = list(keep)[0:100_000]


dataset_filtered = {key: dataset[key][indices] for key in keys}
np.savez_compressed(datafile, **dataset_filtered)

In [ ]:
# big yz plot big figure
fig = plt.figure(figsize=(10,10))
num = 1000
plt.plot(dataset_filtered['y'][0:num].T, dataset_filtered['z'][0:num].T, color='b', alpha=0.1)
plt.show()